In [1]:
import pandas as pd
import re
import pickle
import json
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Load the test data
test_data = pd.read_csv('/home/achoo/Desktop/Honeypot/test_sanitized_logs_combined.csv', delimiter=',', header=None)
test_data.columns = ['eventid', 'src_ip', 'src_port', 'dst_ip', 'dst_port', 'session', 
                     'protocol', 'version', 'hassh', 'hasshAlgorithms', 'message', 
                     'sensor', 'timestamp']

# Define malicious keywords and whitelist phrases as in the original code
malicious_keywords = ['failed', 'whoami', 'uname', 'chattr', 'cat', ' rm', '.ssh', 'authorized_keys',
                      'grep', 'chmod', 'curl', 'not found', 'mkdir', '/bin/', '/tmp/', 'sshd', '.sh', 
                      'ssh-rsa', 'ps', 'crontab', 'uptime', 'ifconfig', 'cpuinfo', 'df', 'chpasswd', 
                      'free', 'pkill', 'pgrep', 'admin']

whitelist_phrases = ["SSH client hassh fingerprint", "New connection"]

# Define the function to flag malicious messages
def flag_malicious(message):
    if any(phrase in message for phrase in whitelist_phrases):
        return 0
    if any(keyword in message for keyword in malicious_keywords):
        return 1
    failed_login_pattern = r'login attempt \[root\/[^\]]+\] failed'
    if re.search(failed_login_pattern, message):
        return 1
    return 0
    
# Apply the flag_malicious function to the 'message' column
test_data['attack'] = test_data['message'].apply(flag_malicious)

# Keep only selected columns
test_data = test_data[['message', 'hasshAlgorithms', 'eventid', 'protocol', 'attack']]

In [2]:
# Initialize the LabelEncoder to encode categorical columns
le = LabelEncoder()

# Encode categorical columns (hasshAlgorithms, eventid, protocol)
test_data['hasshAlgorithms'] = le.fit_transform(test_data['hasshAlgorithms'])
test_data['eventid'] = le.fit_transform(test_data['eventid'])
test_data['protocol'] = le.fit_transform(test_data['protocol'])

# Separate features (X) and labels (y)
X_test = test_data.drop(['attack', 'message'], axis=1)
y_test = test_data['attack']

# Initialize StandardScaler to scale feature data
scaler = StandardScaler()
X_test_scaled = scaler.fit_transform(X_test)

# Load the saved models
with open('finalized_model_XGB.sav', 'rb') as file:
    model_xgb = pickle.load(file)

with open('finalized_model_SVM.sav', 'rb') as file:
    model_svm = pickle.load(file)

with open('finalized_model_RF.sav', 'rb') as file:
    model_rf = pickle.load(file)

with open('finalized_model_Voting.sav', 'rb') as file:
    voting_clf = pickle.load(file)

In [3]:
# Make predictions on the test set using each model
y_pred_xgb = model_xgb.predict(X_test_scaled)
y_pred_svm = model_svm.predict(X_test_scaled)
y_pred_rf = model_rf.predict(X_test_scaled)
y_pred_voting = voting_clf.predict(X_test_scaled)

# Evaluate the models on the test data
print("XGBoost Classifier Evaluation:")
print(f"Accuracy: {accuracy_score(y_test, y_pred_xgb):.4f}")
print(confusion_matrix(y_test, y_pred_xgb))
print(classification_report(y_test, y_pred_xgb))

print("SVM Classifier Evaluation:")
print(f"Accuracy: {accuracy_score(y_test, y_pred_svm):.4f}")
print(confusion_matrix(y_test, y_pred_svm))
print(classification_report(y_test, y_pred_svm))

print("Random Forest Classifier Evaluation:")
print(f"Accuracy: {accuracy_score(y_test, y_pred_rf):.4f}")
print(confusion_matrix(y_test, y_pred_rf))
print(classification_report(y_test, y_pred_rf))

print("Voting Classifier (Soft) Evaluation:")
print(f"Accuracy: {accuracy_score(y_test, y_pred_voting):.4f}")
print(confusion_matrix(y_test, y_pred_voting))
print(classification_report(y_test, y_pred_voting))

# Add predictions from the Voting Classifier as a new column 'attack' in the original data
test_data['attack'] = y_pred_voting

XGBoost Classifier Evaluation:
Accuracy: 0.8051
[[8835  440]
 [1864  681]]
              precision    recall  f1-score   support

           0       0.83      0.95      0.88      9275
           1       0.61      0.27      0.37      2545

    accuracy                           0.81     11820
   macro avg       0.72      0.61      0.63     11820
weighted avg       0.78      0.81      0.77     11820

SVM Classifier Evaluation:
Accuracy: 0.9896
[[9184   91]
 [  32 2513]]
              precision    recall  f1-score   support

           0       1.00      0.99      0.99      9275
           1       0.97      0.99      0.98      2545

    accuracy                           0.99     11820
   macro avg       0.98      0.99      0.98     11820
weighted avg       0.99      0.99      0.99     11820

Random Forest Classifier Evaluation:
Accuracy: 0.7543
[[6809 2466]
 [ 438 2107]]
              precision    recall  f1-score   support

           0       0.94      0.73      0.82      9275
          

In [4]:
# Reload the original test data from the CSV file
original_test_data = pd.read_csv('/home/achoo/Desktop/Honeypot/test_sanitized_logs_combined.csv', delimiter=',', header=None)
original_test_data.columns = ['eventid', 'src_ip', 'src_port', 'dst_ip', 'dst_port', 'session', 
                               'protocol', 'version', 'hassh', 'hasshAlgorithms', 'message', 
                               'sensor', 'timestamp']

# Append the Voting Classifier's results for 'attack' to the original test_data
original_test_data['attack'] = y_pred_voting

# Convert the dataframe to a JSON format
test_data_with_voting_json = original_test_data[['eventid', 'src_ip', 'src_port', 'dst_ip', 'dst_port', 
                                                 'session', 'protocol', 'message', 'sensor', 'timestamp', 
                                                 'attack']].to_json(orient='records', lines=True)

# Save the JSON to a file
with open('/home/achoo/Desktop/Honeypot/votingClasifer_Predictions.json', 'w') as json_file:
    json_file.write(test_data_with_voting_json)
